In [1]:
from model.model import build_model
from data.process import *
import keras

2024-07-07 20:11:48.305592: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-07 20:11:48.313260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 20:11:48.323885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 20:11:48.323916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-07 20:11:48.331785: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_dataframe()


In [3]:

X_train, Y_train = split_input_output(train_df)
print(X_train.shape)
print(Y_train.shape)

(11389751, 11)
(11389751,)


In [ ]:
X_dev, Y_dev = split_input_output(dev_df)
print(X_dev.shape)
print(Y_dev.shape)

In [4]:
train_dataset = convert_to_tensorflow_dataset(X_train, Y_train, 128)
dev_dataset = convert_to_tensorflow_dataset(X_dev, Y_dev, 128)

2024-07-07 20:14:54.306486: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 20:14:54.325952: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 20:14:54.325985: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 20:14:54.328567: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 20:14:54.328602: I external/local_xla/xla/stream_executor

In [5]:
model = build_model(len(X_train.columns), 64, 1)
opt = keras.optimizers.Adam(learning_rate=0.001, weight_decay=0.01)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_dataset, epochs=10, batch_size=64, verbose=1, validation_data=dev_dataset)

Epoch 1/10


I0000 00:00:1720350927.248298   11518 service.cc:145] XLA service 0x7ff8a00209c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720350927.248340   11518 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-07-07 20:15:27.309097: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-07 20:15:27.585172: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


   113/177965 ━━━━━━━━━━━━━━━━━━━━ 4:02 1ms/step - accuracy: 0.8402 - loss: 2.4546 

I0000 00:00:1720350928.962684   11518 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_11', 64 bytes spill stores, 64 bytes spill loads

I0000 00:00:1720350928.964814   11518 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 10875/177965 ━━━━━━━━━━━━━━━━━━━━ 5:54 2ms/step - accuracy: 0.8765 - loss: 1.9888

KeyboardInterrupt: 

In [ ]:
# The history.history["loss"] entry is a dictionary with as many values as epochs that the
# model was trained on. 
df_loss_acc = pd.DataFrame(history.history)
df_loss= df_loss_acc[['loss','val_loss']]
df_loss.rename(columns={'loss':'train','val_loss':'validation'},inplace=True)
df_acc= df_loss_acc[['accuracy','val_accuracy']]
df_acc.rename(columns={'accuracy':'train','val_accuracy':'validation'},inplace=True)
df_loss.plot(title='Model loss',figsize=(12,8)).set(xlabel='Epoch',ylabel='Loss')
df_acc.plot(title='Model Accuracy',figsize=(12,8)).set(xlabel='Epoch',ylabel='Accuracy')

In [ ]:
history.history